In [1]:
!curl localhost:9200

{
  "name" : "0f5b826699fc",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Pe8o2kN0RFGJ4nAF2-cgcA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:


import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)



In [3]:


documents[0]



{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
# q2
# index

In [5]:


from elasticsearch import Elasticsearch



In [6]:
es_client = Elasticsearch('http://localhost:9200') 

In [7]:


index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "rag-course-questions-homework"

es_client.indices.create(index=index_name, body=index_settings)



ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag-course-questions-homework'})

In [8]:


for doc in documents:
    es_client.index(index=index_name, document=doc)



In [9]:
query = "How do execute a command on a Kubernetes pod?"

In [10]:


search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)



In [11]:


result_scores = []
for hit in response['hits']['hits']:
    result_scores.append(hit['_score'])



In [12]:


result_scores



[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

In [26]:

query = "How do copy a file to a Docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)



In [27]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [28]:


for result_doc in result_docs:
    print(result_doc)



{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'mac

In [29]:
for i, q in enumerate(result_docs, 1):
    print(f"Question {i}: {q['question']}")

Question 1: How do I debug a docker container?
Question 2: How do I copy files from my local machine to docker container?
Question 3: How do I copy files from a different folder into docker container’s working directory?


In [42]:

query = "How do copy a file to a Docker container?"


search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

# Search for documents
response = es_client.search(index=index_name, body=search_query)



In [43]:


# Get the 3rd question from the results
third_question = response["hits"]["hits"][2]["_source"]["question"]

print(f"3rd question returned by the search engine: {third_question}")



3rd question returned by the search engine: How do I copy files from a different folder into docker container’s working directory?


In [44]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
"""

context_template = """
Q: {question}
A: {text}
""".strip()

# Initialize the context string
context = ""

# Iterate over the hits in the response to build the context
for doc in response['hits']['hits']:
    context_entry = context_template.format(
        question=doc['_source']['question'],
        text=doc['_source']['text']
    )
    context += context_entry + "\n\n"  # Add double line breaks

# Remove the trailing newlines for clean formatting
context = context.strip()

# Format the prompt with the question and context
prompt = prompt_template.format(question=query, context=context).strip()

# Print the prompt (for debugging purposes)
print(prompt)


You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [45]:


len(prompt)



1446

In [46]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:0000:0100:01


In [48]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [49]:


len(encoding.encode(prompt))



320